In [1]:
import cv2
import csv
import numpy as np
import pandas as pd
import tensorflow as tf

from cv2.typing import MatLike

from non_maximum_suppression import non_max_suppression_fast

print(f'OpenCV: {cv2.__version__}')
print(f'CSV: {csv.__version__}')
print(f'Numpy: {np.__version__}')
print(f'Pandas: {pd.__version__}')
print(f'TensorFlow: {tf.__version__}')

OpenCV: 4.8.1
CSV: 1.0
Numpy: 1.24.3
Pandas: 2.0.3
TensorFlow: 2.15.0


In [2]:
WINDOW = 'Hand Tracking'
PALM_MODEL_PATH = '../models/palm_detection_full.tflite'
ANCHORS_PATH = '../MediaPipeSSDAnchors/anchors.csv'
DETECTION_THRESHOLD = 0.5
BOX_ENLARGE_FACTOR = 1.5
BOX_SHIFT_FACTOR = 0.2

In [3]:
csv_file = open(ANCHORS_PATH, 'r')
anchors = np.r_[
  [x for x in csv.reader(csv_file, quoting=csv.QUOTE_NONNUMERIC)]
]

print(f'Anchors shape: {anchors.shape}')
print(f'Anchors: {anchors[:5]}')

Anchors shape: (2016, 4)
Anchors: [[0.02083333 0.02083333 1.         1.        ]
 [0.02083333 0.02083333 1.         1.        ]
 [0.0625     0.02083333 1.         1.        ]
 [0.0625     0.02083333 1.         1.        ]
 [0.10416667 0.02083333 1.         1.        ]]


In [4]:
interpreter = tf.lite.Interpreter(model_path=PALM_MODEL_PATH)
interpreter.allocate_tensors()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [5]:
input_details = interpreter.get_input_details()

for detail in input_details:
    print(pd.DataFrame.from_dict(detail, orient='index'))

                                                                         0
name                                                               input_1
index                                                                    0
shape                                                     [1, 192, 192, 3]
shape_signature                                           [1, 192, 192, 3]
dtype                                              <class 'numpy.float32'>
quantization                                                      (0.0, 0)
quantization_parameters  {'scales': [], 'zero_points': [], 'quantized_d...
sparsity_parameters                                                     {}


In [6]:
IMG_INPUT_SIZE = tuple(input_details[0]['shape'][1:3])

print(f'Image input size: {IMG_INPUT_SIZE}')

Image input size: (192, 192)


In [7]:
output_details = interpreter.get_output_details()

for detail in output_details:
    print(pd.DataFrame.from_dict(detail, orient='index'))

                                                                         0
name                                                              Identity
index                                                                  279
shape                                                        [1, 2016, 18]
shape_signature                                              [1, 2016, 18]
dtype                                              <class 'numpy.float32'>
quantization                                                      (0.0, 0)
quantization_parameters  {'scales': [], 'zero_points': [], 'quantized_d...
sparsity_parameters                                                     {}
                                                                         0
name                                                            Identity_1
index                                                                  276
shape                                                         [1, 2016, 1]
shape_signature          

In [8]:
def preprocess_img(frame: MatLike):
    # Original Size
    shape = np.r_[frame.shape]

    # Calculate Padding
    pad = (shape.max() - shape[:2]).astype('uint32') // 2

    # Pad Image
    img_pad = np.pad(frame, ((pad[0], pad[0]), (pad[1], pad[1]), (0, 0)), 'constant')

    # Resize Image
    img_small = cv2.resize(img_pad, IMG_INPUT_SIZE)
    
    # Set as contiguous array (speed up)
    img_small = np.ascontiguousarray(img_small)

    img_norm = (img_small / 255).astype('float32')
    
    return img_pad, img_norm, pad

In [9]:
def run_inference(img_norm):
    # Add Batch dimension
    img_input = np.expand_dims(img_norm, axis=0)

    # Set input tensor
    interpreter.set_tensor(input_details[0]['index'], img_input)

    # Run Inference
    interpreter.invoke()

    # Get Output
    """
    output_reg shape is [number of anchors, 18]
    Second dimension 0 - 4 are bounding box offset, width and height: dx, dy, w ,h
    Second dimension 4 - 18 are 7 hand keypoint x and y coordinates: x1,y1,x2,y2,...x7,y7
    """
    # [2016, 18]
    output_reg = interpreter.get_tensor(output_details[0]['index'])[0]

    """
    output_clf shape is [number of anchors]
    it is the classification score if there is a hand for each anchor box
    """
    # [2016, 1]
    output_clf = interpreter.get_tensor(output_details[1]['index'])[0]

    return output_reg, output_clf

In [10]:
def process_output(output_reg, output_clf):
    def _sigm(x):
      return 1 / (1 + np.exp(-x) )
    
    def _get_triangle(self, kp0, kp2, dist=1):
      """get a triangle used to calculate Affine transformation matrix"""

      # 90° rotation matrix used to create the alignment trianlge
      R90 = np.r_[[[0,1],[-1,0]]]

      dir_v = kp2 - kp0
      dir_v /= np.linalg.norm(dir_v)

      dir_v_r = dir_v @ R90.T
      return np.float32([kp2, kp2+dir_v*dist, kp2 + dir_v_r*dist])

    # Get probabilities
    # [2016, 1]
    probabilities = _sigm(output_clf)
    # [2016]: Boolean
    detection_mask = (probabilities > DETECTION_THRESHOLD).flatten()
    # [x,18]
    candidate_detect = output_reg[detection_mask]
    # [x,4]
    candidate_anchors = anchors[detection_mask]
    # [x, 1]
    probabilities = probabilities[detection_mask]

    if candidate_detect.shape[0] == 0:
      return None, None, None
    
    # return candidate_detect, candidate_anchors, probabilities

    # Pick the best bounding box with non maximum suppression
    # the boxes must be moved by the corresponding anchor first
    # [x, 18]
    moved_candidate_detect = candidate_detect.copy()
    moved_candidate_detect[:, :2] = candidate_detect[:, :2] + candidate_anchors[:, :2] * IMG_INPUT_SIZE[0]
    box_ids = non_max_suppression_fast(moved_candidate_detect[:, :4], probabilities)

    if len(box_ids) == 0:
      return None, None, None

    return candidate_detect[box_ids][0], candidate_anchors[box_ids][0], probabilities[box_ids][0]


In [11]:
def get_bbox(candidate_detect, candidate_anchors, shape):
  scale_width = shape[0] / IMG_INPUT_SIZE[0]
  scale_height = shape[1] / IMG_INPUT_SIZE[1]
  bbox = np.zeros((candidate_detect.shape[0], 4))

  cx = candidate_detect[:, 0] + candidate_anchors[:, 0] * shape[0]
  cy = candidate_detect[:, 1] + candidate_anchors[:, 1] * shape[1]
  width = candidate_detect[:, 2] * scale_width
  height = candidate_detect[:, 3] * scale_height

  bbox[:, 0] = cx - width # x_min
  bbox[:, 1] = cy - height # y_min
  bbox[:, 2] = cx + width # x_max
  bbox[:, 3] =  cy + height # y_max

  # Ensure the bounding box coordinates are within the image boundaries
  bbox[:, :2] = np.maximum(0, bbox[:, :2])
  bbox[:, 2:] = np.minimum(shape[1], bbox[:, 2:])

  return bbox

In [12]:
cv2.startWindowThread()
cv2.namedWindow(WINDOW)
capture = cv2.VideoCapture(0)

while capture.isOpened():
  hasFrame, frame = capture.read()
  
  if hasFrame:
    img_pad, img_norm, pad = preprocess_img(frame)
    shape = np.r_[img_pad.shape]
    output_reg, output_clf = run_inference(img_norm)
    candidate_detect, candidate_anchors, probabilities = process_output(output_reg, output_clf)

    if candidate_detect is not None:
      bbox = get_bbox(candidate_detect, candidate_anchors, shape)
      
      # draw box
      for box in bbox:
        cv2.rectangle(img_pad, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (0, 255, 0), 2)

    cv2.imshow(WINDOW, img_pad)
  
  if cv2.waitKey(10) & 0xFF == ord('q'):
    break

capture.release()
cv2.destroyAllWindows()

# Fix for macos bug when closing window
for i in range(4):
    cv2.waitKey(1)